<img src="../../img/dataset_merging.png" alt="Dataset Merging" style="width: 100%; border-radius: 20px;"/>

## Objective
For our project, we received datasets containing bird sightings from 2018-2022 from DDA and Vogelwarte Switzerland. In order to conduct joint analyses on the datasets, they need to be combined. While both datasets fundamentally have the same information and structure, they differ in some aspects. The goal of this notebook is to highlight these aspects and to convert the data using a unified schema to which the data, as well as any future data to be used, can be adapted.


## Scope
In this notebook, we align the Swiss and German dataset of sightings 2018-2022 to create a consistent master dataset. <br><br>
Important decisions we made: <br>
- Remove `estimation_code` feature from the german dataset as it is not given in the swiss dataset and is not considered crucial for our use case.
- Keep `altitude` even though all german entries are zero as the missing altitudes can later be determined based of the location information.
- Align feature names
- Align dtypes
- Align date formats
- Align precisions; drop some precisions in german dataset as they are not in the datafield description and only a minority.
- Align bird species id's and names

## Output data
The resulting master dataset ist stored as *master_bird_data.csv* [here](https://drive.google.com/drive/folders/1R9VHEs6nq8ogPYSSp8IfSbkFWFAoyhm8?usp=sharing).

In [7]:
import pandas as pd

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 0️⃣ Specify your paths </p>

In order to run the notebook, the following datasets are required:
- Swiss dataset: *[birds_ch_2018-2022.csv](https://drive.google.com/drive/folders/1eznk8GyIKt8fPJCb4TVqEIkrNcwonn9m)*
- German dataset: *[birds_de_2018-2022.csv](https://drive.google.com/drive/folders/1eznk8GyIKt8fPJCb4TVqEIkrNcwonn9m)*
- ID translator file that translates german species ID's into ornitho species ID's: *[translation_species_id_germany_vs_ornitho.csv](https://drive.google.com/drive/folders/1VN87gPc_XA212rpyaq2xpJcOSDu8hN5v)* <br>

If you wish to store the resulting master dataset, please specify a target path where it should be stored.

In [23]:
data_path_ch = '../../../01_Data/datasets/birds_ch_2018-2022.csv'  # Provide data path of swiss dataset
data_path_de =  '../../../01_Data/datasets/birds_de_2018-2022.csv'  # Provide data path of german dataset
data_path_translator = '../../../01_Data/translators/translation_species_id_de_vs_ornitho.csv'  # Provide data path of translator file that translates german species id's to ornitho species id's

data_path_master = 'master_bird_data.csv'  # Provide data path where merged dataset shall be saved

#
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 1️⃣ Load data </p>

## Swiss dataset

In [9]:
ch_data = pd.read_csv(data_path_ch, delimiter=';')
ch_data.head()

,ID_SIGHTING,ID_SPECIES,NAME_SPECIES,DATE,TIMING,COORD_LAT,COORD_LON,PRECISION,ALTITUDE,TOTAL_COUNT,ATLAS_CODE_CH,ID_OBSERVER
0,14731644,371.0,Blaumeise,2018-01-21,NaN,46.217211,7.582658,Exakte Lokalisierung,1150,1.0,0,11750.0
1,15360340,361.0,Saatkrähe,2018-03-24,10:41:00,46.923721,7.481304,Exakte Lokalisierung,510,NaN,0,2246.0
2,15360731,358.0,Rabenkrähe,2018-03-24,NaN,46.887983,7.545741,Ort,520,NaN,0,3539.0
3,15360732,495.0,Feldsperling,2018-03-24,NaN,46.887983,7.545741,Ort,520,NaN,0,3539.0
4,15360733,518.0,Buchfink,2018-03-24,NaN,46.887983,7.545741,Ort,520,NaN,0,3539.0


## German dataset

In [10]:
de_data = pd.read_csv(data_path_de)
de_data.head()

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,estimation_code,altitude,total_count,altas_code,beobachter
0,29666944,119,Reiherente,01.01.2018,NaN,53.15776,8.676993,place,EXACT_VALUE,0,24,NaN,37718
1,29666945,141,Gänsesäger,01.01.2018,NaN,53.15776,8.676993,place,EXACT_VALUE,0,1,NaN,37718
2,29666946,24,Kormoran,01.01.2018,04:00,53.15776,8.676993,place,ESTIMATION,0,240,NaN,37718
3,29666947,205,Blässhuhn,01.01.2018,NaN,53.15776,8.676993,place,EXACT_VALUE,0,13,NaN,37718
4,29666948,309,Ringeltaube,01.01.2018,NaN,53.15776,8.676993,place,EXACT_VALUE,0,2,NaN,37718


#
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 2️⃣  Align both dataset to a standardized pattern </p>
The german and swiss dataset have some minor differences in their data structure. In order to merge them into a master dataset, this chapter aligns both to a standardized data pattern.

## 1. Align features
In general, both the german and the swiss datset hold the same features. 

However, the feature `estimation_code` is only present in the swiss dataset and can not be determined in retrospect. It holds information if the bird count is an exact value or an estimation. As discussed with the ornithologists, it is not seen crucial for later modeling purposes and is therefore dropped from the swiss dataset.

In [11]:
de_data.drop(columns='estimation_code', inplace=True)

## 2. Align feature names
Some columns represent the same features but are written in uppercase, or have different names or typos. We use the german column names as default and replace the swiss column names accordingly.

In [12]:
ch_data.columns = ch_data.columns.str.lower()
ch_data.rename({'atlas_code_ch': 'atlas_code'}, axis='columns', inplace=True)
de_data.rename({'beobachter':'id_observer', 'altas_code': 'atlas_code'}, axis='columns', inplace=True)

## 3. Align dtypes
As seen in the table below, the german data uses *float* for `id_species`, `total_count` and `id_observer`, the swiss data uses *int*. <br>
The swiss scheme is used as there are no decimals necessary for these features. The atlas code in the german dataset is already given in *int*, but stored as *object*, since in the german dataset, there are some occurences of str-int-mixed atlas codes (e.g., *A1*). This is a proper atlas code and can stay as-is. 

The other dtypes match.

In [13]:
df = pd.DataFrame(columns=['ch dtype', 'de dtype'])
for col in ch_data.columns:
    df.loc[col] = [ch_data[col].dtype, de_data[col].dtype]
df

,ch dtype,de dtype
id_sighting,int64,int64
id_species,float64,int64
name_species,object,object
date,object,object
timing,object,object
coord_lat,float64,float64
coord_lon,float64,float64
precision,object,object
altitude,int64,int64
total_count,float64,int64


In [15]:
ch_data.id_species = ch_data.id_species.astype('Int64')
ch_data.total_count = ch_data.total_count.astype('Int64')
ch_data.id_observer = ch_data.id_observer.astype('Int64')

de_data.id_species = de_data.id_species.astype('Int64')
de_data.total_count = de_data.total_count.astype('Int64')
de_data.id_observer = de_data.id_observer.astype('Int64')

## 4. Align date format
The swiss data uses *yyyy-mm-dd* <br>
The german data uses *dd.mm.yyyy* <br><br>
The swiss scheme is used as it is more common, the german data format is adapted accordingly. 

In [16]:
def change_dateformat(date):
    d_m_y = date.split('.')
    y_m_d = '{}-{}-{}'.format(d_m_y[2], d_m_y[1], d_m_y[0])
    return y_m_d

de_data.date = de_data.date.apply(change_dateformat)
de_data.head()

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer
0,29666944,119,Reiherente,2018-01-01,NaN,53.15776,8.676993,place,0,24,NaN,37718
1,29666945,141,Gänsesäger,2018-01-01,NaN,53.15776,8.676993,place,0,1,NaN,37718
2,29666946,24,Kormoran,2018-01-01,04:00,53.15776,8.676993,place,0,240,NaN,37718
3,29666947,205,Blässhuhn,2018-01-01,NaN,53.15776,8.676993,place,0,13,NaN,37718
4,29666948,309,Ringeltaube,2018-01-01,NaN,53.15776,8.676993,place,0,2,NaN,37718


## 5. Align precisions
As seen in the table below, the swiss data uses german descriptions for the precision of the coordinates.<br>
Moreover, we can see that some values in the german dataset have very little occurences and are not present in the swiss dataset or in the datafield description provided by the ornithologists.

In [17]:
# Before: Occurences of all precisions in both datasets
print('\033[1m'+'German precision occurences:\n', '\033[0m', de_data.groupby('precision').size())
print('\033[1m'+'\nSwiss precision occurences:\n', '\033[0m', ch_data.groupby('precision').size())

German precision occurences:
  precision
municipality               6
place               10012712
polygone                   4
polygone_precise           8
precise             22903811
square               7186252
subplace                   2
transect_precise          53
dtype: int64

Swiss precision occurences:
  precision
Exakte Lokalisierung    4753055
Kilometerquadrat        3176584
Ort                     1993236
dtype: int64


### Replace swiss descriptions with english counterparts
According to the datafield description by @Johannes, the following upholds:
- *precise* = *Exakte Lokalisierung*
- *square* = *Kilometerquadrat*
- *place* = *Ort*<br><br>

The swiss descriptions are adjusted accordingly to their english counterparts to match the german descriptions.<br>

In [18]:
# Replace swiss precisions with english counterparts
precisions = {'Exakte Lokalisierung': 'precise', 
              'Kilometerquadrat': 'square', 
              'Ort': 'place'}
ch_data.precision = ch_data.precision.map(precisions)

### Drop minority precisions
We also drop the values in the german dataset that have very little occurences and are not present in the swiss dataset and in the datafield description.

In [19]:
# drop all minority precisions in german dataset
precisions_to_drop = 'municipality|polygone|polygone_precise|subplace|transect_precise'
de_data.drop(de_data[de_data.precision.str.contains(precisions_to_drop)].index, inplace=True)

As shown in the table below, we now have aligned and cleaned precisions in both datasets.

In [20]:
# After: Aligned and cleaned precision occurences
print('\033[1m'+'German precision occurences:\n', '\033[0m', de_data.groupby('precision').size())
print('\033[1m'+'\nSwiss precision occurences:\n', '\033[0m', ch_data.groupby('precision').size())

German precision occurences:
  precision
place      10012712
precise    22903811
square      7186252
dtype: int64

Swiss precision occurences:
  precision
place      1993236
precise    4753055
square     3176584
dtype: int64


## 6. Align species ID's
In the german dataset provided to us, the column `id_species` contains german-specific species IDs instead of the ornitho IDs. Using a translation table from ornitho, all species IDs are therefore first translated into their corresponding ornitho IDs.

For example, ID 314 holds different bird species in the german and swiss dataset:

In [21]:
# Example for unmatching id_species in german and swiss dataset
print('Swiss datapoint for ID 314:')
display(ch_data[ch_data.id_species==314].head(1))
print('German datapoint for ID 314:')
display(de_data[de_data.id_species==314].head(1))

Swiss datapoint for ID 314:


,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer
738,15563770,314,Kuckuck,2018-04-16,NaN,46.396284,6.90206,place,370,1,1,725


German datapoint for ID 314:


,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer
5061,29652006,314,Schleiereule,2018-01-01,NaN,52.111929,6.900061,precise,0,1,NaN,94465


We can map the german species IDs to their ornito species IDs.

In [24]:
# Map german-specific species-ID to ornitho species-ID
translator = pd.read_csv(data_path_translator, delimiter=';')
translator_dict = dict(zip(translator.id_species_dbird, translator.id_species_ornitho))
de_data.id_species = de_data.id_species.map(translator_dict).fillna(de_data.id_species)

After the ID replacement, the ID / bird name relation is now correct:

In [25]:
# Example for now matching id_species in german and swiss dataset
print('Swiss datapoint for ID 314:')
display(ch_data[ch_data.id_species==314].head(1))
print('German datapoint for ID 314:')
display(de_data[de_data.id_species==314].head(1))

Swiss datapoint for ID 314:


,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer
738,15563770,314,Kuckuck,2018-04-16,NaN,46.396284,6.90206,place,370,1,1,725


German datapoint for ID 314:


,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer
382861,39272515,314,Kuckuck,2019-04-01,NaN,50.352952,8.674035,precise,0,1,NaN,41754


## 7. Align bird names
In total, 75 bird species have different names in both datasets, usually using a different spelling of 'ss' instead of 'ß'. However, the ID is the same.<br>
Based on the species ID, all swiss bird names are replaced by their german counterpart. If a bird species is only present in the swiss dataset, its name stays as-is.

### Comparison of bird names in Germany and bird names in Switzerland
The table below lists all 75 bird species whose names differ in the german and swiss datasets. <br>

In [26]:
# Create a dicts  with species_id as keys and species_names as values {<species_id>: <species_name>}
german_species_map = dict(zip(de_data.id_species, de_data.name_species))
swiss_species_map = dict(zip(ch_data.id_species, ch_data.name_species))

# Create side-by-side comparison view
species_name_comparison = pd.DataFrame({'German name': pd.Series(german_species_map).sort_index(),
                                        'Swiss name': pd.Series(swiss_species_map).sort_index()}).fillna('-')
differences = species_name_comparison[(species_name_comparison['German name'] != '-') & (species_name_comparison['Swiss name'] != '-') & (species_name_comparison['German name'] != species_name_comparison['Swiss name'])]
differences

,German name,Swiss name
14,Atlantiksturmtaucher (Schwarzschnabelsturmtauc...,Atlantiksturmtaucher
39,Weißstorch,Weissstorch
48,Rotschnabel-Pfeifgans,Rotschnabelpfeifgans
51,Gelbbrust-Pfeifgans,Gelbbrustpfeifgans
55,Schwarzschwan (Trauerschwan),Schwarzschwan
...,...,...
1205,Alexandersittich,Grosser Alexandersittich
1408,"Uferschnepfe (ssp. islandica), Isländische",Uferschnepfe (L.l.islandica)
1477,Schwanzmeise (ssp. caudatus),Schwanzmeise (A.c.caudatus)
1491,Italien-_x_Haussperling,Haussperling x Italiensperling


### Align bird names
Using the german names as default, we replace all swiss names with their german counterpart. If a bird species is only apparent in the swiss dataset, the name is kept as-is.

In [27]:
# In swiss dataset: If a different bird name is used for the same species, replace it with respective german bird name
german_species_map = dict(zip(de_data.id_species, de_data.name_species))
ch_data.name_species = ch_data.id_species.map(german_species_map).fillna(ch_data.name_species)

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 3️⃣  Merge datasets </p>
The two standardized datasets are now merged. In order to later distinguish between the German and Swiss data, we append the column `country`. <br>

In [28]:
ch_data['country'] = 'ch'
de_data['country'] = 'de'
master_data = pd.concat([de_data, ch_data])

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 4️⃣ Save Master Dataset</p>

Lastly, the dataset is saved as a CSV file at the specified target file path.

In [ ]:

master_data.to_csv(data_path_master)
master_data.head()

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer,country
0,29666944,119,Reiherente,2018-01-01,NaN,53.15776,8.676993,place,0,24,NaN,37718,de
1,29666945,141,Gänsesäger,2018-01-01,NaN,53.15776,8.676993,place,0,1,NaN,37718,de
2,29666946,24,Kormoran,2018-01-01,04:00,53.15776,8.676993,place,0,240,NaN,37718,de
3,29666947,205,Blässhuhn,2018-01-01,NaN,53.15776,8.676993,place,0,13,NaN,37718,de
4,29666948,310,Ringeltaube,2018-01-01,NaN,53.15776,8.676993,place,0,2,NaN,37718,de
